In [97]:
import pandas as pd
import numpy as np


In [98]:
!pwd

/grid/kinney/home/desmara/MPSA_pred/SOTA_models/head_interpretation/SAE_Hackathon/data/whole_genome


In [99]:
gene_split = pd.read_csv('./gene_split.csv', index_col=[0])
display(gene_split.head())
gene_data = pd.read_csv('./gene_data.csv', index_col=[0])
display(gene_data.head())



,gene,Test_set,CV0,CV1,CV2,CV3,CV4
0,ENSG00000285542,False,False,False,True,False,False
1,ENSG00000132294,False,True,False,False,False,False
2,ENSG00000016490,False,True,False,False,False,False
3,ENSG00000167182,False,True,False,False,False,False
4,ENSG00000168758,False,True,False,False,False,False


,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type
Gene stable ID,,,,,,
ENSG00000284532,17,28360654,28360734,1,MIR4723,miRNA
ENSG00000238933,9,61642383,61642494,1,RF00019,misc_RNA
ENSG00000275693,10,88992370,88992539,1,RF02116,misc_RNA
ENSG00000275451,15,62343029,62343138,1,MIR6085,miRNA
ENSG00000222870,7,94495299,94495404,1,RNU6-1328P,snRNA


In [100]:
gene_data

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type
Gene stable ID,,,,,,
ENSG00000284532,17,28360654,28360734,1,MIR4723,miRNA
ENSG00000238933,9,61642383,61642494,1,RF00019,misc_RNA
ENSG00000275693,10,88992370,88992539,1,RF02116,misc_RNA
ENSG00000275451,15,62343029,62343138,1,MIR6085,miRNA
ENSG00000222870,7,94495299,94495404,1,RNU6-1328P,snRNA
...,...,...,...,...,...,...
ENSG00000285013,CHR_HSCHR7_3_CTG4_4,144396023,144396316,1,AC243837.3,protein_coding
ENSG00000285154,CHR_HSCHR7_3_CTG4_4,144414782,144415702,1,AC243837.4,protein_coding
ENSG00000284933,CHR_HSCHR7_3_CTG4_4,144457568,144474951,1,ARHGEF5,protein_coding


In [101]:
only_split_genes = gene_data[gene_data.index.isin(gene_split.gene)]
only_split_genes

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type
Gene stable ID,,,,,,
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding
ENSG00000275049,CHR_HSCHR15_5_CTG8,82403250,82412947,-1,AC011295.1,protein_coding
ENSG00000276804,CHR_HSCHR19LRC_LRC_J_CTG3_1,54956665,54980661,-1,NLRP7,protein_coding
ENSG00000276565,CHR_HSCHR11_1_CTG6,1586202,1591699,1,AP006285.3,protein_coding
ENSG00000281024,CHR_HG2030_PATCH,133356700,133361317,1,SURF2,protein_coding
...,...,...,...,...,...,...
ENSG00000284827,CHR_HSCHR7_3_CTG4_4,144387992,144388921,1,AC243837.1,protein_coding
ENSG00000285013,CHR_HSCHR7_3_CTG4_4,144396023,144396316,1,AC243837.3,protein_coding
ENSG00000285154,CHR_HSCHR7_3_CTG4_4,144414782,144415702,1,AC243837.4,protein_coding


In [102]:
np.unique(only_split_genes['Chromosome/scaffold name'], return_counts=True)

(array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
        '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9',
        'CHR_HG107_PATCH', 'CHR_HG126_PATCH', 'CHR_HG1311_PATCH',
        'CHR_HG1342_HG2282_PATCH', 'CHR_HG1362_PATCH',
        'CHR_HG142_HG150_NOVEL_TEST', 'CHR_HG151_NOVEL_TEST',
        'CHR_HG1708_PATCH', 'CHR_HG1815_PATCH', 'CHR_HG1832_PATCH',
        'CHR_HG1_PATCH', 'CHR_HG2002_PATCH', 'CHR_HG2021_PATCH',
        'CHR_HG2023_PATCH', 'CHR_HG2030_PATCH', 'CHR_HG2046_PATCH',
        'CHR_HG2047_PATCH', 'CHR_HG2057_PATCH', 'CHR_HG2058_PATCH',
        'CHR_HG2063_PATCH', 'CHR_HG2066_PATCH', 'CHR_HG2067_PATCH',
        'CHR_HG2072_PATCH', 'CHR_HG2088_PATCH', 'CHR_HG2095_PATCH',
        'CHR_HG2104_PATCH', 'CHR_HG2114_PATCH', 'CHR_HG2116_PATCH',
        'CHR_HG2191_PATCH', 'CHR_HG2213_PATCH', 'CHR_HG2217_PATCH',
        'CHR_HG2232_PATCH', 'CHR_HG2233_PATCH', 'CHR_HG2235_PATCH',
        'CHR_HG2236_PATCH', 'CHR_HG2239_PATCH', 'CHR_HG2247_PATCH',
 

In [103]:
only_split_genes.index.unique()

Index(['ENSG00000275568', 'ENSG00000275049', 'ENSG00000276804',
       'ENSG00000276565', 'ENSG00000281024', 'ENSG00000278695',
       'ENSG00000278232', 'ENSG00000274672', 'ENSG00000281185',
       'ENSG00000274566',
       ...
       'ENSG00000285328', 'ENSG00000285075', 'ENSG00000285105',
       'ENSG00000285408', 'ENSG00000284912', 'ENSG00000284827',
       'ENSG00000285013', 'ENSG00000285154', 'ENSG00000284933',
       'ENSG00000285114'],
      dtype='object', name='Gene stable ID', length=22889)

In [104]:
to_chunk = pd.merge(left=only_split_genes, right=gene_split, left_index=True, right_on='gene', how='inner')
to_chunk.rename(columns={'gene': 'Gene stable ID'}, inplace=True)
to_chunk.set_index('Gene stable ID', inplace=True)
to_chunk

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type,Test_set,CV0,CV1,CV2,CV3,CV4
Gene stable ID,,,,,,,,,,,,
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False
ENSG00000275049,CHR_HSCHR15_5_CTG8,82403250,82412947,-1,AC011295.1,protein_coding,False,False,True,False,False,False
ENSG00000276804,CHR_HSCHR19LRC_LRC_J_CTG3_1,54956665,54980661,-1,NLRP7,protein_coding,False,False,False,False,False,True
ENSG00000276565,CHR_HSCHR11_1_CTG6,1586202,1591699,1,AP006285.3,protein_coding,False,False,False,False,True,False
ENSG00000281024,CHR_HG2030_PATCH,133356700,133361317,1,SURF2,protein_coding,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000284827,CHR_HSCHR7_3_CTG4_4,144387992,144388921,1,AC243837.1,protein_coding,False,True,False,False,False,False
ENSG00000285013,CHR_HSCHR7_3_CTG4_4,144396023,144396316,1,AC243837.3,protein_coding,False,False,False,True,False,False
ENSG00000285154,CHR_HSCHR7_3_CTG4_4,144414782,144415702,1,AC243837.4,protein_coding,False,False,True,False,False,False


In [105]:
def chunker(start, stop, window):
    j=None
    for i in range(start, stop, window):
        if j is not None:
            yield((j, i-1))
            j=i
        else:
            j=i

    if stop-j < window:
        yield(j,stop)
    else:
        mid = stop-(stop-j)//2
        yield(j,mid-1)
        yield(mid,stop)

def chunkify(to_chunk, chunk_size=5000):
    to_chunk['chunks'] = to_chunk.apply(lambda x: list(chunker(x['Gene start (bp)'],x['Gene end (bp)'], chunk_size)), axis=1)
    to_chunk = to_chunk.explode('chunks')
    to_chunk[['ChunkStart', 'ChunkEnd']] = pd.DataFrame(to_chunk['chunks'].tolist(), index=to_chunk.index)
    return(to_chunk)

gene_chunks = chunkify(to_chunk)
gene_chunks.head()

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type,Test_set,CV0,CV1,CV2,CV3,CV4,chunks,ChunkStart,ChunkEnd
Gene stable ID,,,,,,,,,,,,,,,
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32695437, 32700436)",32695437,32700436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32700437, 32705436)",32700437,32705436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32705437, 32710436)",32705437,32710436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32710437, 32715436)",32710437,32715436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32715437, 32720246)",32715437,32720246


In [106]:
(gene_chunks['ChunkStart']<gene_chunks['ChunkEnd']).all()

np.True_

In [107]:
gene_chunks['ChunkEnd']-gene_chunks['ChunkStart']

Gene stable ID
ENSG00000275568    4999
ENSG00000275568    4999
ENSG00000275568    4999
ENSG00000275568    4999
ENSG00000275568    4809
                   ... 
ENSG00000285114    4999
ENSG00000285114    4999
ENSG00000285114    4999
ENSG00000285114    4999
ENSG00000285114    3713
Length: 298157, dtype: int64

In [108]:
gene_chunks['AdjustedEnd'] = gene_chunks['ChunkEnd']+4999 - (gene_chunks['ChunkEnd']-gene_chunks['ChunkStart'])
gene_chunks

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type,Test_set,CV0,CV1,CV2,CV3,CV4,chunks,ChunkStart,ChunkEnd,AdjustedEnd
Gene stable ID,,,,,,,,,,,,,,,,
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32695437, 32700436)",32695437,32700436,32700436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32700437, 32705436)",32700437,32705436,32705436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32705437, 32710436)",32705437,32710436,32710436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32710437, 32715436)",32710437,32715436,32715436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32715437, 32720246)",32715437,32720246,32720436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285114,CHR_HSCHR8_7_CTG7,129757225,129795938,-1,GSDMC,protein_coding,False,False,True,False,False,False,"(129772225, 129777224)",129772225,129777224,129777224
ENSG00000285114,CHR_HSCHR8_7_CTG7,129757225,129795938,-1,GSDMC,protein_coding,False,False,True,False,False,False,"(129777225, 129782224)",129777225,129782224,129782224
ENSG00000285114,CHR_HSCHR8_7_CTG7,129757225,129795938,-1,GSDMC,protein_coding,False,False,True,False,False,False,"(129782225, 129787224)",129782225,129787224,129787224


In [109]:
((gene_chunks['AdjustedEnd']-gene_chunks['ChunkStart'])==4999).all()

np.True_

In [116]:
gene_chunks['FlankStart'] = gene_chunks['ChunkStart']-5000
gene_chunks['FlankEnd'] = gene_chunks['AdjustedEnd']+5000
print(((gene_chunks['FlankEnd']-gene_chunks['FlankStart'])==14999).all())
gene_chunks

True


,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type,Test_set,CV0,CV1,CV2,CV3,CV4,chunks,ChunkStart,ChunkEnd,AdjustedEnd,FlankStart,FlankEnd
Gene stable ID,,,,,,,,,,,,,,,,,,
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32695437, 32700436)",32695437,32700436,32700436,32690437,32705436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32700437, 32705436)",32700437,32705436,32705436,32695437,32710436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32705437, 32710436)",32705437,32710436,32710436,32700437,32715436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32710437, 32715436)",32710437,32715436,32715436,32705437,32720436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32715437, 32720246)",32715437,32720246,32720436,32710437,32725436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285114,CHR_HSCHR8_7_CTG7,129757225,129795938,-1,GSDMC,protein_coding,False,False,True,False,False,False,"(129772225, 129777224)",129772225,129777224,129777224,129767225,129782224
ENSG00000285114,CHR_HSCHR8_7_CTG7,129757225,129795938,-1,GSDMC,protein_coding,False,False,True,False,False,False,"(129777225, 129782224)",129777225,129782224,129782224,129772225,129787224
ENSG00000285114,CHR_HSCHR8_7_CTG7,129757225,129795938,-1,GSDMC,protein_coding,False,False,True,False,False,False,"(129782225, 129787224)",129782225,129787224,129787224,129777225,129792224


In [117]:
gene_chunks.to_csv('./gene_chunks.csv')

In [118]:
test_chunks = gene_chunks[gene_chunks.Test_set]
test_chunks.to_csv('./test_chunks.csv')

In [119]:
test_chunks

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type,Test_set,CV0,CV1,CV2,CV3,CV4,chunks,ChunkStart,ChunkEnd,AdjustedEnd,FlankStart,FlankEnd
Gene stable ID,,,,,,,,,,,,,,,,,,
ENSG00000163352,1,154993586,154994315,1,LENEP,protein_coding,True,False,False,False,False,False,"(154993586, 154994315)",154993586,154994315,154998585,154988586,155003585
ENSG00000203667,1,244835322,244845057,1,COX20,protein_coding,True,False,False,False,False,False,"(244835322, 244840321)",244835322,244840321,244840321,244830322,244845321
ENSG00000203667,1,244835322,244845057,1,COX20,protein_coding,True,False,False,False,False,False,"(244840322, 244845057)",244840322,244845057,244845321,244835322,244850321
ENSG00000173627,1,183646404,183653316,-1,APOBEC4,protein_coding,True,False,False,False,False,False,"(183646404, 183651403)",183646404,183651403,183651403,183641404,183656403
ENSG00000173627,1,183646404,183653316,-1,APOBEC4,protein_coding,True,False,False,False,False,False,"(183651404, 183653316)",183651404,183653316,183656403,183646404,183661403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000187258,7,34658239,34878332,1,NPSR1,protein_coding,True,False,False,False,False,False,"(34868239, 34873238)",34868239,34873238,34873238,34863239,34878238
ENSG00000187258,7,34658239,34878332,1,NPSR1,protein_coding,True,False,False,False,False,False,"(34873239, 34878238)",34873239,34878238,34878238,34868239,34883238
ENSG00000187258,7,34658239,34878332,1,NPSR1,protein_coding,True,False,False,False,False,False,"(34878239, 34878332)",34878239,34878332,34883238,34873239,34888238


In [120]:
val_chunks = gene_chunks[gene_chunks.CV0]
val_chunks.to_csv('./val_chunks.csv')
train_chunks = gene_chunks[~gene_chunks.Test_set & ~gene_chunks.CV0]
train_chunks.to_csv('./train_chunks.csv')


In [121]:
train_chunks

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type,Test_set,CV0,CV1,CV2,CV3,CV4,chunks,ChunkStart,ChunkEnd,AdjustedEnd,FlankStart,FlankEnd
Gene stable ID,,,,,,,,,,,,,,,,,,
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32695437, 32700436)",32695437,32700436,32700436,32690437,32705436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32700437, 32705436)",32700437,32705436,32705436,32695437,32710436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32705437, 32710436)",32705437,32710436,32710436,32700437,32715436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32710437, 32715436)",32710437,32715436,32715436,32705437,32720436
ENSG00000275568,CHR_HSCHR15_4_CTG8,32695437,32720246,1,ARHGAP11A,protein_coding,False,False,False,False,True,False,"(32715437, 32720246)",32715437,32720246,32720436,32710437,32725436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285114,CHR_HSCHR8_7_CTG7,129757225,129795938,-1,GSDMC,protein_coding,False,False,True,False,False,False,"(129772225, 129777224)",129772225,129777224,129777224,129767225,129782224
ENSG00000285114,CHR_HSCHR8_7_CTG7,129757225,129795938,-1,GSDMC,protein_coding,False,False,True,False,False,False,"(129777225, 129782224)",129777225,129782224,129782224,129772225,129787224
ENSG00000285114,CHR_HSCHR8_7_CTG7,129757225,129795938,-1,GSDMC,protein_coding,False,False,True,False,False,False,"(129782225, 129787224)",129782225,129787224,129787224,129777225,129792224
